In [19]:
import os
import requests
import pandas as pd
from pathlib import Path

from notebook_context import *  # gives you project_root, paths, etc.

# ⚠️ For now you can paste your key here, but long-term it's nicer to use an env var.
BALLDONTLIE_API_KEY = os.getenv("BALLDONTLIE_API_KEY", "c5ae7df3-682e-450c-b47e-f7e91396379e")

if BALLDONTLIE_API_KEY == "c5ae7df3-682e-450c-b47e-f7e91396379e":
    print("⚠️ Remember to set BALLDONTLIE_API_KEY or paste your key in this variable.")
else:
    print("Balldontlie API key loaded.")
    
print("Project root:", project_root)


⚠️ Remember to set BALLDONTLIE_API_KEY or paste your key in this variable.
Project root: C:\Users\wdors\qepc_project


In [20]:
BASE_URL = "https://api.balldontlie.io/v1"

def _headers():
    return {"Authorization": BALLDONTLIE_API_KEY}

def fetch_all_teams():
    """Fetch all NBA teams from Balldontlie."""
    teams = []
    cursor = None

    while True:
        params = {"per_page": 100}
        if cursor is not None:
            params["cursor"] = cursor

        resp = requests.get(f"{BASE_URL}/teams", headers=_headers(), params=params, timeout=10)
        resp.raise_for_status()
        js = resp.json()

        teams.extend(js.get("data", []))

        cursor = js.get("meta", {}).get("next_cursor")
        if not cursor:
            break

    teams_df = pd.DataFrame(teams)
    print(f"Fetched {len(teams_df)} teams from Balldontlie.")
    return teams_df


def fetch_all_player_injuries():
    """Fetch all current player injuries (paginated)."""
    injuries = []
    cursor = None

    while True:
        params = {"per_page": 100}
        if cursor is not None:
            params["cursor"] = cursor

        resp = requests.get(f"{BASE_URL}/player_injuries", headers=_headers(), params=params, timeout=10)
        resp.raise_for_status()
        js = resp.json()

        data = js.get("data", [])
        injuries.extend(data)

        cursor = js.get("meta", {}).get("next_cursor")
        if not cursor:
            break

    print(f"Fetched {len(injuries)} injury records from Balldontlie.")
    return injuries


In [21]:
def fetch_all_player_injuries():
    """Fetch all current player injuries (paginated).
    Returns a list; if unauthorized, returns [] and prints a warning.
    """
    injuries = []
    cursor = None

    while True:
        params = {"per_page": 100}
        if cursor is not None:
            params["cursor"] = cursor

        resp = requests.get(
            f"{BASE_URL}/player_injuries",
            headers=_headers(),
            params=params,
            timeout=10,
        )

        if resp.status_code == 401:
            print("⚠️ Balldontlie: 401 Unauthorized on /player_injuries.")
            print("   This usually means your current plan doesn't include this endpoint.")
            print("   QEPC will skip live injuries from Balldontlie and rely on other sources.")
            return []

        resp.raise_for_status()
        js = resp.json()

        data = js.get("data", [])
        injuries.extend(data)

        cursor = js.get("meta", {}).get("next_cursor")
        if not cursor:
            break

    print(f"Fetched {len(injuries)} injury records from Balldontlie.")
    return injuries


In [22]:
out_path = project_root / "data" / "Injury_Overrides_live_balldontlie.csv"
inj_live_df.to_csv(out_path, index=False)
print("Saved live injury overrides to:", out_path)


Saved live injury overrides to: C:\Users\wdors\qepc_project\data\Injury_Overrides_live_balldontlie.csv


In [23]:
base_dir = project_root / "data"

paths = [
    base_dir / "Injury_Overrides_live_balldontlie.csv",  # NEW: from Balldontlie
    base_dir / "Injury_Overrides_live_espn.csv",         # if you keep using ESPN at all
    base_dir / "Injury_Overrides_data_driven.csv",       # your impact model
    base_dir / "Injury_Overrides.csv",                   # manual / long-term base
]

frames = []
for p in paths:
    if p.exists():
        print("Including injuries from:", p.name)
        df = pd.read_csv(p)
        df["__source"] = p.name
        frames.append(df)

if frames:
    injuries = pd.concat(frames, ignore_index=True)
    print("Total injury rows loaded:", len(injuries))
else:
    injuries = None
    print("⚠️ No injury override files found.")


Including injuries from: Injury_Overrides_live_balldontlie.csv


EmptyDataError: No columns to parse from file